<a href="https://colab.research.google.com/github/paulodowd/EMATM0053_21_22/blob/main/L1_Moving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 1: Moving

In this labsheet we will use some basic exercises to familiarise ourselves with how to move the simulated e-puck robot.



## Writing your first Controller



### The Simulator Architecture

Before we begin writing our first controller, it is useful to first understand the general premise of how the Webots simulation operates.  The below image illustrates the `architecture` of the simulator:

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_SimulatorController.png?raw=true">
<br>
</p>

In the above, we can conceptually seperate the operation of the simulator versus the robot controller.  Because we are simulating a process over time, it is necessary to "step" (advance) the simulator `discretely` (at fixed intervals).  When the simulator is stepped, the sensors are updated for the new environment status, and the actuators are provided their new instructions.  


<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_UpdateCycle.png?raw=true">
<br>
</p>

Whilst it is possible to step the simulator forwards within any part of your code (as illustrated below), it is extremely likely you will develop bugs and erroneous behaviours in your simulator robot.  This is because time will be able to advance from any part of your program, and there will be a loss of any general synchronisation to your program.  

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_WrongSimStep.png?raw=true">
<br>
</p>

Instead, it is better to write a controller to not interfere with how time is incremented  - similar to the way real time advances regardless of our human actions.  Therefore, you should write your controller so that the `loop()` function returns at every time interval, and for your controller to re-evaluate present circumstances at every time interval.  


This is not especially convenient, however, it is much closer to how we would develop code for an embedded computing platform such as a microcontroller.  

<hr><br><br><br><br>

### Setting Motor Velocities

Reading the template code provided, we can find the following lines of code within `setup()` which set the velocities of the robot motors:

```c
  wb_motor_set_velocity(left_motor, 0.0);
  wb_motor_set_velocity(right_motor, 0.0);
```

These two lines set a 0 velocity to the left and right motor respectively, so that when the robot activates it's default action is not to move.  We can see that a Webots built-in function (prefixed with `wb_`) is used.  We can see that `wb_motor_set_velocity()` takes two arguments seperated by a comma.  

The first argument is the `handle` used to interface with the motor device in the simulation environment (e.g. used to read or write data).  All the device handles you will need have been setup for your already.  These handles were created as global variables with the special type `WbDeviceTag`, and then assigned the device in the preceeding lines of code:

```c
  // Setup motors
  left_motor = wb_robot_get_device("left wheel motor");
  right_motor = wb_robot_get_device("right wheel motor");
```

After this setup routine, setting the motor velocity is as simple as using `wb_motor_set_velocity()` whenever needed, providing the correct device handle, and providing the second numeric value argument.  You will see a similar patten when you use other devices in the future, such as the sensors.

Therefore, utilising:
```c
  wb_motor_set_velocity(left_motor, 1.0);
```

Sets a forward velocity to the left motor.  And utilising:
```c
  wb_motor_set_velocity(right_motor, -1.0);
```

Sets a reverse velocity to the right motor.  

**Importantly**, the robot will only receive the last instruction when `wb_robot_step(TIME_STEP)` is called from within `main()`.  As noted int he previous section, it might be tempting to call `wb_robot_step(TIME_STEP)` within your controller to advance time - but you shouldn't do this!   If we look at the following:

```c
  wb_motor_set_velocity(right_motor, 0.1);
  // ...
  wb_motor_set_velocity(right_motor, 0.5);
  // ... 
  wb_motor_set_velocity(right_motor, -0.1);
  // ...
  wb_motor_set_velocity(right_motor, 0.0);
```

In the above example, although 4 different motor commands were instructed, only the last would be used when `wb_robot_step(TIME_STEP)` is activated.  Therefore, within this time frame, the velocity of the right motor would be set to 0 (all other instructions ignored), and the consequences then computed.

<hr><br><br><br><br>

### Time

Within the template controller code provided (`labsheet_x.c`) we can also find the following line of code:

```c
  // Report current time.
  printf("Loop at %.2f (secs)\n", wb_robot_get_time() );
```

and when you run the controller, you should see something similar to the following output in the bottom-most `console` pane:

```c
Loop at 0.03 (secs)     
...
```
You should be able to observe that the time reported is advancing as the simulation progresses.  Therefore, your simulated robot has access to the amount of time that has passed since it was initially accessed.  On a microcontroller, we can create this functionality by configuring a `timer peripheral`, as discussed in the lecture series.  

In the above code extract, the built-in function `wb_robot_get_time()` is called within another function, `printf()`.  In which case,  `wb_robot_get_time()` returns a floating point value, which `printf` then integrates into the `string` at the location marked `%.2f`.  This is then displayed in the console pane.  

You can use this same built-in function to get the latest time elsewhere in your code, such as in the following examples:

```c
// e.g., within a selection statement:
if( wb_robot_get_time() < 10.0 ) {

}

// e.g., by storing the value to use later:
float t;
t = wb_robot_get_time();

// etc.
```

Note that, just like reading the sensors or instructing the actuators, the time reported will not advance until your loop() function returns, and the main thread calls `wb_robot_step(TIME_STEP)`.

<hr><br><br><br><br>

## Exercise 1: Motor Velocity (2hours)

1. Conduct some research: what type of motor does the real e-puck use to rotate it's wheels?  
 - what are some advantages to this type of motor?
 - what are some disadvantages to this type of motor?

2. Explore: Write a controller to have your robot move fowards continuously.

3. Explore: Write a controller to have your robot move backwards continuously.

4. Explore: Find the maximum and minimum velocities which can be used with the motors.

5. Solve: Write a controller to increment a positive motor velocity on each simulation time step to maximum velocity, and then to decrement the motor velocity back to zero at each time step.
 - tip: create a variable to store the last motor velocity used.  
 - tip: it might be useful to create a variable to store whether you are currently incrementing or decrementing the motor velocity.
 - tip: remember to consider `local` and `global` scope of variables.

6. Explore: Write a controller to have your robot rotate on the spot (i.e. no forward or backward movement).
 - which pair of values cause the robot to rotate left?
 - which pair values cause the robot to rotate right?

7. Solve: Find the best motor velocities to have your robot move on an arc, to trace the same half-circle curve of the provided line following map. 
 





## Exercise 2: Time (3hours)

1. When your (simulated or real) robot reads the time that has passed since it was activated, is this **`exteroceptive`** or **`proprioceptive`** information?

2. Solve: Use `wb_robot_get_time()` to program your robot to drive forwards for the first 4 seconds of simulation, and then stop.

3. Solve: Write some code so that your robot drives forwards for 2 seconds, then stops for 2 seconds, and can repeat this procedure endlessly. 
  - **help**: to implement this, considering capturing the **`elapsed time`** (or difference in time, `dt`).  You can use `global variables` to operate as `time stamps` - a variable to record the time value on specific events.
  - **help**: it is useful to write a function to operate as a timer to return a `true` or `false` value if a requested period of time has elapsed.

4. Solve: Adjust the code for (3) so that your robot drives forwards for 2 seconds, and then turns right for 2 seconds, and repeats this procedure endlessly.
  - Can you adjust your code so that the robot coordinates its motion to create square pattern?
  - Can you adjust the time values so that your robot can trace the square starting box on the line following map provided?
  - Alternatively, adjust the time values so that your robot can move around one of the square obstacles without colliding.

5. Are these exercises implementing **`closed-loop`** or **`open-loop`** control?

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_NavigateSquare.png?raw=true">
<br>
</p>